In [1]:
import pandas as pd
import numpy as np

## Введение

#### Для построения геомеханической модели необходимы исходные каротажные данные, которые записываются в стволе скважины. 
#### На этих данных базируется расчет упруго-прочностных свойств и дальнейшее получение напряженно-деформированного состояния горного массива.
#### Минимально необходимым комплексом каротажных данных является: акустические данные (продольная и поперечная волна в виде скорости или медленности), плотностной каротаж и гамма-каротаж. 

## Проблематика

#### Отсутствие записи поперечной волны на большинстве скважин из-за дорогостоящих исследований.
#### В большинстве скважин при это имеется записть продольной волны и плотности, которые имеют высокую корреляцию с поперечной волной.
#### Геомеханики вручную строят линейную регрессию на имеющихся данных и восстаналивают недостающий каротаж.
#### Необходимо оптимизировать этот процесс и сократить время выполнения однообразной задачи восстановления.

## Цель проекта: Обучить модель восстановливать поперечную волну на основе имеющихся данных по поперечной волне и плотности

#### Примечание: 
#### Две модели с возможностью инкрементального обучения: 
#### 1. Восстановление от продольной волны и плотности
#### 2. Восстановление от продольной волны при отсутствии каротажа плотности

## 1. Обзор данных

#### Нам представлены данные для первого региона, где нужно реализовать восстановление поперечной волны на основе каротажей плотности и продольной волны для дальнейших скважин.
#### В наличии 5 скважин, 4 из которых буду представлять тренировочный набор данных, а 1 скважина будет являться тестовой для предсказания модели.
#### Скважины имеют 4 столбца: ...

In [6]:
region_1 = [
    "region_1_well_1.csv",
    "region_1_well_2.csv",
    "region_1_well_3.csv",
    "region_1_well_4.csv"
]
# Объединеные скважин в один тренировочный набор данных
df = [pd.read_csv(file) for file in region_1]
train = pd.concat(df, ignore_index=True)

train.to_csv("train_region_1.csv", index=False)

# Создение тестового датасета
test = pd.read_csv('region_1_well_5_test.csv')

In [7]:
train.head(5)

,MD,DTCO,DTSM,RHOB
0,m,us/m,us/m,g/cm3
1,MD,DT_full,DTS_full,RHOB_full_1
2,0,-999.25,-999.25,-999.25
3,0.1524,-999.25,-999.25,-999.25
4,0.3048,-999.25,-999.25,-999.25


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157492 entries, 0 to 157491
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   MD      157492 non-null  object
 1   DTCO    157492 non-null  object
 2   DTSM    157492 non-null  object
 3   RHOB    118119 non-null  object
dtypes: object(4)
memory usage: 4.8+ MB


In [9]:
print(f'Количество пропущенных значений:\n{train.isnull().sum()}')

Количество пропущенных значений:
MD          0
DTCO        0
DTSM        0
RHOB    39373
dtype: int64


#### Первые две строки следует удалить - это единицы измерения и названия каротажей, выгруженные из рабочих проектов
#### Так как вы соединили несколько файлов, подобные строки лежат еще внутри датафрейма
#### Избавимся от них при переходе к числовому типу, вставив NaN

In [11]:
train = train.apply(pd.to_numeric, errors='coerce')
test = test.apply(pd.to_numeric, errors='coerce')

In [12]:
print(f'Количество пропущенных значений:\n{train.isnull().sum()}')

Количество пропущенных значений:
MD          8
DTCO        8
DTSM        8
RHOB    39379
dtype: int64


#### Удалим NaN, которые остались от названий единиц измеренеия и каротажей

In [14]:
train = train.dropna(subset=['DTCO'])

In [15]:
print(f'Количество пропущенных значений:\n{train.isnull().sum()}')

Количество пропущенных значений:
MD          0
DTCO        0
DTSM        0
RHOB    39371
dtype: int64


#### Значения -999.25 означают отсутствие каротажа на данной глубине, их следует удалить

In [17]:
train = train.loc[~(train == -999.25).any(axis=1)]
test = test.loc[~(test == -999.25).any(axis=1)]

#### Теперь можем удалить столбец глубины, так как в дальнейшем он нам не понадобится

In [19]:
train = train.drop("MD", axis=1) 
test = test.drop("MD", axis=1) 

In [20]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30344 entries, 9317 to 145354
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DTCO    30344 non-null  float64
 1   DTSM    30344 non-null  float64
 2   RHOB    17193 non-null  float64
dtypes: float64(3)
memory usage: 948.2 KB


#### Видим пропущенные значения в каротаже плотности. Каротаж плотности, как и каротаж продольной волны, обладает высокой корреляцией с поперечной волной.  
#### Заполнить средним или медианой в данном случае не получится, так как оба признака необходимы для наиболее корректного восстановления DTSM.
#### Предлагается обучить две модели: восстановление DTSM только от DTCO и восстановление DTSM от DTCO и RHOB.
#### В таком случае в дальнейшем человек сможет восстаналивать каротаж как при наличии одного каротажа, так и при наличии обоих каротажей. 

#### Разделим данные на два датасета 

In [23]:
train_dtco_rhob = train.dropna(subset=["RHOB"]) # Датасет с плотностью
train_dtco = train.drop("RHOB", axis=1) # Датасет без плотности

test_dtco_rhob = test.dropna(subset=["RHOB"])
test_dtco = test.drop("RHOB", axis=1)

## 2. Разведочный анализ данных